In [3]:
#Codigo obtenido a partir del libro Generative Adversarial Networks with Python Deep Learning Generative Models for Image Synthesis and Image Translation (Jason Brownlee).
#Librerias
from tensorflow.keras.applications.inception_v3 import InceptionV3,preprocess_input
from skimage.transform import resize
from scipy.linalg import sqrtm
import numpy as np
import os
import glob

from PIL import Image

In [4]:
#Modelo InceptionV3 para obtener características
model = InceptionV3(include_top=False,pooling='avg', input_shape=(128,128,3))

In [5]:
#Función que reescala todas las imágenes al tamaño deseado.
def scale_images(images,new_shape):
    images_list = list()
    for image in images:
        new_image= resize(image,new_shape,0)

        images_list.append(new_image)
    return np.asarray(images_list)

In [6]:
#Función que encapsula el cálculo del FID
def calculate_fid(model,images1,images2):
    act1 = model.predict(images1)
    act2 = model.predict(images2)

    mu1, sigma1 = act1.mean(axis=0), np.cov(act1,rowvar=False)
    mu2, sigma2 = act2.mean(axis=0), np.cov(act2,rowvar=False)

    ssdiff = np.sum((mu1-mu2)**2.0)

    covmean = sqrtm(sigma1.dot(sigma2))

    if np.iscomplexobj(covmean):
        covmean = covmean.real
    
    fid = ssdiff + np.trace(sigma1 + sigma2 - 2.0 * covmean)
    return fid

In [7]:
#Función que carga todas las imágenes de una ruta.
def load_images(path):
    files = glob.glob(path)

    im_list = [Image.open(img) for img in files]

    arr_list = [np.array(img) for img in im_list]

    images = np.stack(arr_list)
    return images

In [15]:
images1 = load_images('C:/stylegan/CycleGAN/fidGen/maleNoGl/*.png')
images2 = load_images('C:/stylegan/CycleGAN/fidGen/vsYoungMale/*.png')



images1 = images1.astype('float32')
images2 = images2.astype('float32')

images1 = scale_images(images1,(128,128,3))
images2 = scale_images(images2,(128,128,3))

images1 = preprocess_input(images1)
images2 = preprocess_input(images2)

fid = calculate_fid(model,images1,images2)




37/37 [==============================] - 1s 19ms/step


In [20]:
print(fid)

292.5738825183353
